# CAO Project
---

This notebook contains:
 -  An overview of how to load CAO points information from the CAO website
 -  A detailed comparison of CAO points in 2019, 2020, and 2021
 -  Appropriate plots and other visualisations of the data

## Background on the CAO

The Central Applications Office (CAO), founded in January 1976, is the organisation responsible for overseeing undergraduate applications to colleges and universities in the Republic of Ireland<sup>1</sup>. 
Undergraduates apply for entry into university through the CAO website and not through individual universities. The CAO is a points based system, with students getting the highest points securing offers from universities.

### Leaving Certificate Points
The points obtained from the leaving certificate exam determine what course the student is eligible for. Points for higher and ordinary level shown below<sup>2</sup>.

***Higher Level Points***

| Higher Level Grade | Points |
| -- | -- |
| H1 | 100|
| H2 | 88 |
| H3 | 77 |
| H4 | 66 |
| H5 | 56 |
| H6 | 46 |
| H7 | 37 |
| H8 | 0 |

***Ordinary Level Points***

| Ordinary Level Grade | Points |
| -- | -- |
| O1 | 56 |
| O2 | 46 |
| O3 | 37 |
| O4 | 28 |
| O5 | 20 |
| O6 | 12 |
| O7 | 0 |
| O8 | 0 |

A students points are calculated based on their six best subjects for a maximum score of 625, if the student takes and passes higher level maths<sup>2</sup>.

## CAO Points Analysis


------

### Importing the Required Libraries

In [1]:
# Regualar expressions
import re

# Convient HTTP requests
import requests as rq

# Dates and times
import datetime as dt

import os

# Data frames
import pandas as pd

# For downloading
import urllib.request as urlrq

# Nice plots
import seaborn as sns

# Numpy
import numpy as np

### Saving the datasets

It is best practice to save the dataset each time the data is ran, this ensures if the data on the websites is changed then the dataset will reflect these changes. In order for each save to not overwrite the previous version datetime is used.

In [2]:
# Get current date and time
now = dt.datetime.now()

# Format as a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

<br>

## CAO 2021 Points
-----

The [2021 course points](http://www2.cao.ie/points/l8.php) for level 8 courses.

```Resquests``` allows you to send HTTP/1.1 requests extremely easily<sup>3</sup>. ```resp``` is the response object from which we get all the information we need<sup>4</sup>. The HTTP ```<Respoonse [200]>``` success status response code indicates that the request has succeeded<sup>5</sup>.

In [3]:
# Use resp.text to see the text
resp = rq.get('http://www2.cao.ie/points/l8.php')

# peak
resp

<Response [200]>

In [4]:
# Create a filepath for the original data
# Always sort year, month, date
pathhtml= 'data/CAO2021_' + nowstr + '.html'

#### Error on Server

Technically, the server says we should decode as per:
> Content-Type: text/html; charset=iso-8859-1

However one line uses \x96 which isn't defined in iso-8859-1
Therefore we use the similar decoding standard cp1252.

utf-8 usual decode but since Irish uses fadas you must decode using iso-8859-1 so it's not written in utf-8. Inspect the webpage and then it shows you the source code and network. In network headers you can see that the character set used is iso-8859-1.

In [5]:
# Server uses wrong encoding
original_encoding = resp.encoding

# Change to cp1252
resp.encoding = 'cp1252'

In [6]:
# Save original html file
with open(pathhtml, 'w') as f:
    f.write(resp.text)

#### Regular Expressions

Regular expressions can be used to clean the dataset. These work by searching and matching using a sequence if characters that the user specifies. Regular expressions can be used to remove rows/spaces/whitespaces that are not required. Below is the regular expression used to clean this dataset.
```(r'([A-Z]{2}[0-9]{3})  (.*)')```
The first part of this code ```(r'(``` denotes a raw string, followed by ```[A-Z]{2}``` which means any two capital letters between A and Z, followed by ```[0-9]{3}``` which looks for 3 numbers between 0 and 9 immediately after the two letters and finally ```(.*)')``` this looks for a wildcard (an unknown number of any character).

In [7]:
#Compile the regular expression for matching lines
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)')

#### Creating the filepath


In [8]:
# The file path for the csv file.
path2021 = 'data/cao2021_csv_' + nowstr + '.csv'

In [9]:
# Keep track of how many courses we process.
no_lines=0

# Open the csv file for writing.
with open(path2021, 'w') as f:
    #Write a header row
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    for line in resp.iter_lines():
        dline = line.decode('cp1252')
        # Match only the lines we want - the ones representing courses.
        #if re.fullmatch('[A-Z]{2}[0-9]{3}  .*[0-9]{3}(\*)? *', line.decode('iso-8859-1')):
        if re_course.fullmatch(dline):
            no_lines = no_lines + 1
            #print(line)
            #csv_version = re_course.sub(r'\1,\2,\3\4', dline)
            #print(csv_version)
            #linesplit = re.split('  +', dline)
            # debug
            #print(len(linesplit), linesplit, dline)
            course_code = dline[:5]
            #print(course_code)
            course_title = dline[7:57].strip()
            # Round one points
            course_points = re.split(' +', dline[60:])
            if len(course_points) !=2:
            #print(f"'{course_title}'")
                course_points = course_points[:2]
            # join fields using a comma
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            f.write(','.join(linesplit) + '\n')
            #print(','.join(linesplit))
            
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


<br>

**NB**: it was verified as of 13 Nov 2021 that there were 949 courses exactly in the CAO 2021 points list

In [10]:
header_list=['Code','Title', 'Points R1', 'Points R2']
df2021 = pd.read_csv(path2021, encoding='cp1252',names=header_list, skiprows=1)

In [11]:
df2021

,Code,Title,Points R1,Points R2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


<br>

## CAO 2020 Points

The [CAO points for 2020](https://www.cao.ie/index.php?page=points&p=2020) level 8 courses.

----

The CAO website does not offer each years data in the same format the data for 2020 courses is only available in an excel spreadsheet. However due to the inbuilt excel reader in pandas this makes it easier for us to create a CSV file.

As with the 2021 data, first the data is saved using the timestamp.

In [12]:
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

In [13]:
pathxlsx = 'data/CAO2020_' + nowstr + '.xlsx'

In [14]:
urlrq.urlretrieve(url2020, pathxlsx)

('data/CAO2020_20211128_162711.xlsx',
 <http.client.HTTPMessage at 0x7fb7b01b16d0>)

In [15]:
# Download and parse the excel
df2020 = pd.read_excel(url2020, skiprows = 10)

In [16]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Spot check random row
df2020.iloc[753]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Road Transport Technology and Management
COURSE CODE2                                                           LC286
R1 POINTS                                                                264
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      264
EOS Random *                                                             NaN
EOS Mid-point                                                            360
LEVEL                                                                      7
HEI                                         Limerick Institute of Technology
Test/Interview #                                                         NaN

In [18]:
# Spot check last row
df2020.iloc[1463]
# can also use -1 to get the very last row, -2 second last row......

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [19]:
path2020 = 'data/CAO2020_' + nowstr + '.csv'

In [20]:
df2020.to_csv(path2020)

<br>

## 2019 Points

The [CAO points for 2019](https://www.cao.ie/index.php?page=points&p=2019) for level 8 courses.

----

Again the data offered from the CAO site changes for 2019, for this year it's given in a pdf. This means the data was cleaned by hand as it is the quickest way to clean data gleaned from a pdf. Below are the steps I used to clean the data. Note as I use iOS there are some differences in how the data was converted to a csv file.

##### Steps to reproduce

1. Download original pdf file
2. Open original pdf file with preview
3. Copy all to Microsoft Word
4. Save Microsoft Word's converted pdf in docx format
5. Re-save Word document for editing
6. Delete headers and Footers
7. Delete preamble at the start
8. Opened with vs code
9. Removed blank lines and college provider
10. Used find and replace to remove tabs and replace with comma
11. Used find and replace to remove all ` with '

In [21]:
df2019 = pd.read_csv('data/CAO2019_20211113_105625_edited.csv', sep='\t')

In [22]:
df2019

,Code,Title,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328
1,AL802,Software Design with Cloud Computing,301,306
2,AL803,Software Design with Mobile Apps and Connected...,309,337
3,AL805,Network Management and Cloud Infrastructure,329,442
4,AL810,Quantity Surveying,307,349
...,...,...,...,...
925,WD200,Arts (options),221,296
926,WD210,Software Systems Development,271,329
927,WD211,Creative Computing,275,322
928,WD212,Recreation and Sport Management,274,311


**Note**: As 2019 data does not offer Round 1 and Round 2 data this will make analysing the three dataframes less accurate.

<br>

## Concat and Join
***

In [23]:
courses2021 = df2021[['Code', 'Title']]

In [24]:
courses2021

,Code,Title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
944,WD211,Creative Computing
945,WD212,Recreation and Sport Management
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [25]:
courses2020 = df2020[['COURSE CODE2', 'COURSE TITLE']]
courses2020.columns = ['Code', 'Title']

In [26]:
courses2020

,Code,Title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1459,WD208,Manufacturing Engineering
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


In [27]:
courses2019 = df2019[['Code', 'Title']]
courses2019

,Code,Title
0,AL801,Software Design with Virtual Reality and Gaming
1,AL802,Software Design with Cloud Computing
2,AL803,Software Design with Mobile Apps and Connected...
3,AL805,Network Management and Cloud Infrastructure
4,AL810,Quantity Surveying
...,...,...
925,WD200,Arts (options)
926,WD210,Software Systems Development
927,WD211,Creative Computing
928,WD212,Recreation and Sport Management


In [28]:
allcourses = pd.concat([courses2021, courses2020, courses2019], ignore_index=True)

allcourses

,Code,Title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
3338,WD200,Arts (options)
3339,WD210,Software Systems Development
3340,WD211,Creative Computing
3341,WD212,Recreation and Sport Management


In [29]:
allcourses.reset_index()

,index,Code,Title
0,0,AL801,Software Design for Virtual Reality and Gaming
1,1,AL802,Software Design in Artificial Intelligence for...
2,2,AL803,Software Design for Mobile Apps and Connected ...
3,3,AL805,Computer Engineering for Network Infrastructure
4,4,AL810,Quantity Surveying
...,...,...,...
3338,3338,WD200,Arts (options)
3339,3339,WD210,Software Systems Development
3340,3340,WD211,Creative Computing
3341,3341,WD212,Recreation and Sport Management


In [30]:
allcourses.sort_values('Code')

,Code,Title
175,AC120,International Business
949,AC120,International Business
2581,AC120,International Business
950,AC137,Liberal Arts
2582,AC137,Liberal Arts
...,...,...
2412,WD230,Mechanical and Manufacturing Engineering
946,WD230,Mechanical and Manufacturing Engineering
3342,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [31]:
allcourses.loc[175]['Title']

'International Business'

In [32]:
allcourses.loc[949]['Title']

'International Business'

In [33]:
# Finds all extra copies of duplicated rows
allcourses[allcourses.duplicated()]

,Code,Title
949,AC120,International Business
950,AC137,Liberal Arts
952,AD102,Graphic Design and Moving Image Design (portfo...
955,AD204,Fine Art (portfolio)
956,AD211,Fashion Design (portfolio)
...,...,...
3338,WD200,Arts (options)
3339,WD210,Software Systems Development
3340,WD211,Creative Computing
3341,WD212,Recreation and Sport Management


In [34]:
# Dataframe with duplicates removed
allcourses.drop_duplicates()

,Code,Title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
3281,TL801,Animation Visual Effects and Motion Design
3282,TL802,"TV, Radio and New Media"
3283,TL803,Music Technology
3286,TL812,Computing with Digital Media


In [35]:
# Finds all extra copies of duplicated rows
allcourses[allcourses.duplicated(subset=['Code'])]

,Code,Title
949,AC120,International Business
950,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
952,AD102,Graphic Design and Moving Image Design (portfo...
953,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
3338,WD200,Arts (options)
3339,WD210,Software Systems Development
3340,WD211,Creative Computing
3341,WD212,Recreation and Sport Management


In [36]:
# Dataframe with duplicates removed - based only on code
allcourses.drop_duplicates(subset=['Code'], inplace=True, ignore_index=True)

In [37]:
allcourses.set_index('Code', inplace=True)

In [38]:
allcourses

,Title
Code,
AL801,Software Design for Virtual Reality and Gaming
AL802,Software Design in Artificial Intelligence for...
AL803,Software Design for Mobile Apps and Connected ...
AL805,Computer Engineering for Network Infrastructure
AL810,Quantity Surveying
...,...
SG441,Environmental Science
SG446,Applied Archaeology
TL803,Music Technology


<br>

### Join to the Points
-----

In [39]:
# Set index to code column
df2021.set_index('Code', inplace=True)
df2021.columns = ['Title','points_R1_2021','points_R2_2021']
df2021

,Title,points_R1_2021,points_R2_2021
Code,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN
AL802,Software Design in Artificial Intelligence for...,313,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,NaN
AL805,Computer Engineering for Network Infrastructure,321,NaN
AL810,Quantity Surveying,328,NaN
...,...,...,...
WD211,Creative Computing,270,NaN
WD212,Recreation and Sport Management,262,NaN
WD230,Mechanical and Manufacturing Engineering,230,230


In [40]:
allcourses = allcourses.join(df2021[['points_R1_2021','points_R2_2021']])
allcourses

,Title,points_R1_2021,points_R2_2021
Code,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN
AL802,Software Design in Artificial Intelligence for...,313,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,NaN
AL805,Computer Engineering for Network Infrastructure,321,NaN
AL810,Quantity Surveying,328,NaN
...,...,...,...
SG441,Environmental Science,NaN,NaN
SG446,Applied Archaeology,NaN,NaN
TL803,Music Technology,NaN,NaN


In [41]:
df2020_r1 = df2020[['COURSE CODE2', 'R1 POINTS','R2 POINTS']]
df2020_r1.columns = ['Code', 'points_R1_2020', 'points_R2_2020']
df2020_r1

,Code,points_R1_2020,points_R2_2020
0,AC120,209,NaN
1,AC137,252,NaN
2,AD101,#+matric,NaN
3,AD102,#+matric,NaN
4,AD103,#+matric,NaN
...,...,...,...
1459,WD208,188,NaN
1460,WD210,279,NaN
1461,WD211,271,NaN
1462,WD212,270,NaN


In [42]:
df2020_r1.set_index('Code', inplace=True)
df2020_r1

,points_R1_2020,points_R2_2020
Code,,
AC120,209,NaN
AC137,252,NaN
AD101,#+matric,NaN
AD102,#+matric,NaN
AD103,#+matric,NaN
...,...,...
WD208,188,NaN
WD210,279,NaN
WD211,271,NaN


In [43]:
allcourses = allcourses.join(df2020_r1)
allcourses

,Title,points_R1_2021,points_R2_2021,points_R1_2020,points_R2_2020
Code,,,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN,303,NaN
AL802,Software Design in Artificial Intelligence for...,313,NaN,332,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,NaN,337,NaN
AL805,Computer Engineering for Network Infrastructure,321,NaN,333,NaN
AL810,Quantity Surveying,328,NaN,319,NaN
...,...,...,...,...,...
SG441,Environmental Science,NaN,NaN,NaN,NaN
SG446,Applied Archaeology,NaN,NaN,NaN,NaN
TL803,Music Technology,NaN,NaN,NaN,NaN


In [44]:
df2019_r1 = df2019[['Code', 'EOS','Mid']]

df2019_r1

,Code,EOS,Mid
0,AL801,304,328
1,AL802,301,306
2,AL803,309,337
3,AL805,329,442
4,AL810,307,349
...,...,...,...
925,WD200,221,296
926,WD210,271,329
927,WD211,275,322
928,WD212,274,311


In [45]:
df2019_r1.set_index('Code', inplace=True)

In [46]:
allcourses = allcourses.join(df2019_r1)

In [47]:
allcourses

,Title,points_R1_2021,points_R2_2021,points_R1_2020,points_R2_2020,EOS,Mid
Code,,,,,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN,303,NaN,304,328
AL802,Software Design in Artificial Intelligence for...,313,NaN,332,NaN,301,306
AL803,Software Design for Mobile Apps and Connected ...,350,NaN,337,NaN,309,337
AL805,Computer Engineering for Network Infrastructure,321,NaN,333,NaN,329,442
AL810,Quantity Surveying,328,NaN,319,NaN,307,349
...,...,...,...,...,...,...,...
SG441,Environmental Science,NaN,NaN,NaN,NaN,297,358
SG446,Applied Archaeology,NaN,NaN,NaN,NaN,289,290
TL803,Music Technology,NaN,NaN,NaN,NaN,264,288


<br>

## Plotting the Data
***

In [48]:
allcourses

,Title,points_R1_2021,points_R2_2021,points_R1_2020,points_R2_2020,EOS,Mid
Code,,,,,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN,303,NaN,304,328
AL802,Software Design in Artificial Intelligence for...,313,NaN,332,NaN,301,306
AL803,Software Design for Mobile Apps and Connected ...,350,NaN,337,NaN,309,337
AL805,Computer Engineering for Network Infrastructure,321,NaN,333,NaN,329,442
AL810,Quantity Surveying,328,NaN,319,NaN,307,349
...,...,...,...,...,...,...,...
SG441,Environmental Science,NaN,NaN,NaN,NaN,297,358
SG446,Applied Archaeology,NaN,NaN,NaN,NaN,289,290
TL803,Music Technology,NaN,NaN,NaN,NaN,264,288


# References
<sup>1</sup>[Central Applications Office](https://en.wikipedia.org/wiki/Central_Applications_Office)  
<sup>2</sup>[Irish Leaving Certificate Examination Points](https://www.cao.ie/index.php?page=scoring&s=lcepointsgrid)  
<sup>3</sup>[Requests:HTTP for Humans](https://docs.python-requests.org/en/latest/)  
<sup>4</sup>[Requests:Quickstart](https://docs.python-requests.org/en/latest/user/quickstart/)  
<sup>5</sup>[200 OK](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/200)  